#### All imports here:

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Reading data set

In [2]:
# For the purpose of the project, I imported the file from my google drive while using google colab.
# I have added the dataset file 'yahoo_dataset.csv' in the github repo.
# However, if you still wish to access the file from your google drive, then
# you can execute the following code on google colab signed into the same account
# as the google drive that you wish to access.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [56]:
# Load the dataset
train_data = pd.read_csv('/content/drive/MyDrive/infinite-data/history.csv')

<ipython-input-56-1220a43f522c>:2: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/content/drive/MyDrive/infinite-data/history.csv')


In [23]:
train_data = pd.read_csv("./datasets/history.csv")
train_data.head()

/var/folders/gk/qw9z66_x6f568n5ry11rsp980000gn/T/ipykernel_50635/600742029.py:1: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("./datasets/history.csv")


,id,type_code,is_registered,country_code,currency_code,is_active,class_id,debit_code,last_trade_date,contract_type,...,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,CASH SWEEP,f,NaN,CAD,t,3.0,0,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,1268094,RRIF,t,NaN,CAD,t,5.0,C,2022-07-04,18.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 0:00,BATCH,t,Churn
2,606613,SPOUSAL RRSP,t,NaN,CAD,t,5.0,A,2018-07-05,16.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 0:00,GUERINO,f,Churn
3,741930,CASH,f,NaN,CAD,t,3.0,T,NaN,12.0,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,1137922,CASH,f,NaN,CAD,t,3.0,T,NaN,17.0,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,NaN,NaN,f,Churn


In [24]:
info_df = pd.read_csv("account_data_info.csv")
pd.set_option('display.max_colwidth', None)
info_df

,Columns,Description,Field Values
0,type_code,Account Type,CSH - Cash/RSP (note: Registered accounts are account type CSH; plan type is determined by PLAN IND)
1,is_registered,NaN,MRL - Margin - Long
2,country_code,NaN,MRS - Margin - Short
3,currency_code,Currency of the Accoun,COD - Cash on Delivery
4,is_active,NaN,COM - Commodity
...,...,...,...
106,updated_at,NaN,NaN
107,is_olob,NaN,NaN
108,retail_last_maintenance_time,NaN,NaN
109,retail_last_maintenance_user,NaN,NaN


#### Dropping Columns and Research

In [25]:
# Aditya Parekh:
# I am removing these columns because they have 600k+ NULL values.
rem_cols_na = ['country_code', 'cashflows_custody_fee', 'arp_pension_origin', 'sss_location']
# I am removing these columns because they have 0 NULL values, but only one unique value.
rem_cols_one_uniq = ['net_of_fees', 'fee_paid_separately', 'custody_fee_withdrawal', 'is_fee_exempt']
# The following columns are NULL when 'class_id' = NULL. DON'T REMOVE THESE YET.
null_3198_cols = ['debit_code', 'last_trade_date', 'contract_type', 'inception_date',
             'branch', 'credit_limit_type', 'retail_plan', 'language_code', 'dividend_confirm_code']

In [26]:
# Aditya Mani:
# 1. esir_number: Contains 673327 missing values and cannot be imputed (has very little unique values)
# 2. shareholder_language: Contains 623554 missing values. Says whether the shareholder speaks English or French and there seems to be an equal proportion.
# 3. has_no_min_commission: No Null values and only 1 unique value.
# 4. rep_commission_rate: 1 unique value and few missing values that cannot be imputed.
# 5. spousal_age_flag: No Null values and only 1 unique value.
# 6. is_parameters_account: No Null values and only 1 unique value.
# 7. rrsp_limit_reached: No Null values and only 1 unique value.
# 8. is_portfolio_account: No Null values and only 1 unique value.
# 9. sss_type: Lot of missing values
# 10. sss_agent: Lot of missing values
# 11. target_grantor_grantee_flag: Missing values
# 12. terminal_code: Missing values

to_drop = ['esir_number', 'shareholder_language', 'has_no_min_commission', 'rep_commission_rate',
           'spousal_age_flag', 'is_parameters_account', 'rrsp_limit_reached', 'is_portfolio_account',
             'sss_type', 'sss_agent', 'target_grantor_grantee_flag', 'terminal_code']

missing_3198 = ['options_trading_type',
 'is_midwest_clearing_account',
 'rep_commission_override',
 'interest_dividend_conversion_type',
 'guarantee_gtor_type',
 'deceased_fair_market_value',
 'iso_funds_code']

In [27]:
# Jithin Krishna:

# Has too many null values
# last_update_date is the maintaineance date and was deemed not relevant to the churn
drop_cols = ["special_tag", "conjunction", "title", "function_code" ,"tms_settlement_location", "loan_limit_override", "last_update_date"]

# These values have 1 unique value and is not useful and hence shall be dropped
drop_uniq_cols = ["portfolio_cost_method", "portfolio_name_address_option", "portfolio_summary_option","interactive_portfolio_code"]

drop_3198_cols = ["portfolio_report_option", "mailing_consent"]

In [28]:
# Srishti Prayag:
# remove columns with more than 300,000 missing values
too_many_nans = ['special_fee_code', 'resp_specimen_plan', 'plan_end_date', 'retail_last_maintenance_user']

# list of columns with exactly 316985 missing values; may be connected so don't drop yet
nan_316985_columns = ['is_pledged', 'non_calendar_year_end', 'plan_effective_date', 'is_resp', 'number_of_beneficiaries', 'rrif_original_date', 'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp', 'retail_last_maintenance_time']

# list of columns with only 1 unique value
only_one_unique = ['is_plan_grandfathered', 'is_gl_account', 'is_control_account']

In [29]:
rem_cols = []
rem_cols.extend(rem_cols_na)
rem_cols.extend(rem_cols_one_uniq)
rem_cols.extend(to_drop)
rem_cols.extend(drop_cols)
rem_cols.extend(drop_uniq_cols)
rem_cols.extend(too_many_nans)
rem_cols.extend(only_one_unique)
train_data = train_data.drop(rem_cols, axis=1)

In [30]:
# Drop rows where column 'class_id' is NaN
train_data = train_data.dropna(subset=['class_id'])

In [10]:
train_data.to_csv("./cleaned_data.csv", index=False)

In [31]:
# Srishti & Adi - Imputations/Deletions
# 1. retail_last_maintenance_time, non_calendar_year_end: Drop the column
# 2. is_resp, is_hrdc_resp, retail_plan: Group by type_code and fill the missing values with the mode of the group
# 3. is_family_resp: If type_code is RESP and number_of_beneficiaries is greater than 1, then fill with 't' else 'f'
# 4. use_original_date_for_payment_calc - if RESP in type_code then t, else f 
# 5. rrif_original_date, plan_effective_date: Fill with inception_date (default)
# 6. number_of_beneficiaries: Fill with 0 (assuming no beneficiaries)
# 7. is_pledged, non_plan_book_value: Fill with N (default)
# Code:
train_data = train_data.drop(['retail_last_maintenance_time', 'non_calendar_year_end'], axis=1)
train_data['is_resp'] = train_data.groupby('type_code')['is_resp'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'f'))
train_data['is_hrdc_resp'] = train_data.groupby('type_code')['is_hrdc_resp'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'f'))
train_data['is_family_resp'] = np.where((train_data['type_code'] == 'RESP') & (train_data['number_of_beneficiaries'] > 1), 't', 'f')
train_data['retail_plan'] = train_data.groupby('type_code')['retail_plan'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'N/A'))
train_data['use_original_date_for_payment_calc'] = np.where(train_data['type_code'] == 'RESP', 't', 'f')
train_data['rrif_original_date'] = train_data['rrif_original_date'].fillna(train_data['inception_date'])
train_data['number_of_beneficiaries'] = train_data['number_of_beneficiaries'].fillna(0.0)
train_data['plan_effective_date'] = train_data['plan_effective_date'].fillna(train_data['inception_date'])
train_data['is_pledged'] = train_data['is_pledged'].fillna('N')
train_data['non_plan_book_value_flag'] = train_data['non_plan_book_value_flag'].fillna('N')
train_data.head()

,id,type_code,is_registered,currency_code,is_active,class_id,debit_code,last_trade_date,contract_type,inception_date,...,number_of_beneficiaries,rrif_original_date,use_original_date_for_payment_calc,is_family_resp,is_hrdc_resp,inserted_at,updated_at,is_olob,visible_in_reports,label
0,893724,CASH SWEEP,f,CAD,t,3.0,0,NaN,NaN,2017-08-18,...,0.0,2017-08-18,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,f,Churn
1,1268094,RRIF,t,CAD,t,5.0,C,2022-07-04,18.0,2018-11-08,...,0.0,2018-11-08,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,t,Churn
2,606613,SPOUSAL RRSP,t,CAD,t,5.0,A,2018-07-05,16.0,2009-04-08,...,0.0,2009-04-08,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,f,Churn
3,741930,CASH,f,CAD,t,3.0,T,NaN,12.0,2021-01-04,...,0.0,2021-01-04,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,t,Churn
4,1137922,CASH,f,CAD,t,3.0,T,NaN,17.0,2017-07-21,...,0.0,2017-07-21,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,f,Churn


In [ ]:
# Aditya & Jithin - Imputations/Deletions

In [32]:
# Missing Values Check
missing_values_train = train_data.isnull().sum()
value_counts = train_data.nunique()
missing_values_train = pd.concat([missing_values_train, value_counts], axis=1)
missing_values_train.columns = ['Missing Values', 'Value Counts']
missing_values_train = missing_values_train[missing_values_train['Missing Values'] > 0]
missing_values_train = missing_values_train.sort_values(by='Missing Values', ascending=False)
missing_values_train

,Missing Values,Value Counts
is_spousal,313447,2
is_arp_locked,313447,2
last_trade_date,304677,5027
risk_tolerance,142332,847
investment_objective,142255,1471
has_discrete_auth,95798,1
contract_type,92625,19
discretionary_trading_authorized,75876,2
last_maintenance_time,340,1734
type_code,237,19
